In [77]:
import pandas as pd
import numpy as np

from itertools import chain
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import GRU, MaxPooling1D, Embedding
from keras.models import Model
from keras import layers, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [79]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

ImportError: cannot import name 'conditional'

In [105]:

#Import Training and Testing Data
tr

combined = pd.concat([train,test],sort=True)
len(combined)

49159

### Tokenization

In [106]:
tokenized_single_posts = [nltk.tokenize.word_tokenize(i) for i in combined['tweet']]
len(tokenized_single_posts)

49159

In [107]:
leng = []
for i in range(len(tokenized_single_posts)):
    length = len(tokenized_single_posts[i])
    leng.append(length)

### stopwords removal

In [108]:
stp_removed = []

for i in range (len(tokenized_single_posts)):
    stp = [word for word in tokenized_single_posts[i] if word not in stopwords.words('english')]
    stp_removed.append(stp)
    

In [109]:
len(stp_removed)

49159

### Lemmatize the stop removed posts

In [110]:
words_lemma = []
lemma = nltk.WordNetLemmatizer()
for i in range(len(stp_removed)):
    words = [lemma.lemmatize(word) for word in stp_removed[i]]
    words_lemma.append(words)

In [111]:
len(words_lemma)

49159

### Remove all digit words

In [112]:
words_noNum = []
for i in range(len(words_lemma)):
    words = [word for word in words_lemma[i] if word.isdigit() == False]
    words_noNum.append(words)

In [113]:
len(words_noNum)

49159

### Remove single character words

In [114]:
words_nonSingle = []
for i in range(len(words_noNum)):
    words = [word for word in words_noNum[i] if len(word) > 1]
    words_nonSingle.append(words)

In [115]:
len(words_nonSingle)

49159

### Remove non-alphabetic words

In [116]:
words_alpha = []
for i in range(len(words_nonSingle)):
    words = [word for word in words_nonSingle[i] if word.isalpha()]
    words_alpha.append(words)

In [117]:
len(words_alpha)

49159

In [ ]:
combined['words_count'] = [len(i) for i in words_alpha]

In [122]:
noun_freq = []
verb_freq = []
adjective_freq = []
adverb_freq = []
for i in range(len(words_alpha)):
    word_pos_tag = nltk.pos_tag(words_alpha[i])
    count_noun = 0
    count_verb = 0
    count_adjective = 0
    count_adverb = 0
    for j in range(len(word_pos_tag)):
        if word_pos_tag[j][1] == "NN":
            count_noun += 1
        if word_pos_tag[j][1] == 'VB':
            count_verb += 1
        if word_pos_tag[j][1] == 'JJ':
            count_adjective += 1
        if word_pos_tag[j][1] == 'RB':
            count_adverb += 1
    noun_freq.append(count_noun/(len(words_alpha[i]) + 1))
    verb_freq.append(count_verb/(len(words_alpha[i])+1))
    adjective_freq.append(count_adjective/(len(words_alpha[i])+1))
    adverb_freq.append(count_adverb/(len(words_alpha[i])+1))

In [123]:
freq_dict = {'noun_freq' : noun_freq, 'verb_freq' : verb_freq, 'adjective_freq' : adjective_freq, 'adverb_freq' : adverb_freq}

In [ ]:
combined = combined.join(pd.DataFrame(freq_dict))

In [125]:
combined.head()

,label,tweet,adjective_freq,adverb_freq,noun_freq,verb_freq
id,,,,,,
1,0.0,@user when a father is dysfunctional and is s...,0.066667,0.066667,0.400000,0.066667
2,0.0,@user @user thanks for #lyft credit i can't us...,0.000000,0.000000,0.666667,0.000000
3,0.0,bihday your majesty,0.000000,0.000000,0.600000,0.200000
4,0.0,#model i love u take with u all the time in ...,0.000000,0.250000,0.500000,0.000000
5,0.0,factsguide: society now #motivation,0.250000,0.000000,0.500000,0.000000


#### read the GLoVE 100 file for preparing an Embedding layer of 100 dimenions

In [97]:
GLOVE_DIR = '/home/pallavi/'
MAX_SEQUENCE_LENGTH = 50
MAX_NUM_WORDS = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

embeddings_index = {}

with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


#### Initialize the Embedding layer with pre-trained weights from the GLoVE model:

In [126]:
## Define the sequence lengths, max number of words and embedding dimensions
MAX_SEQUENCE_LENGTH = 50   # Sequence length of each sentence. If more, crop. If less, pad with zeros
MAX_NB_WORDS = 10000        # Top 10000 frequently occuring words

#### Train-validation-test split. We separate validation data to use early stopping callback feature during training our model below

In [188]:
train  = combined.iloc[:31962 ,:]
# train=x_train.drop(['tweet'],axis=1)
test = combined.iloc[31962:,:]
# test=x_test.drop(['tweet','label'],axis=1)

features = ['tweet', 'words_count', 'adjective_freq', 'noun_freq', 'adverb_freq', 'verb_freq']
labels = ['label']
x_train_val = train.loc[:,features]
y_train_val = train.loc[:,labels]

In [189]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, 
                                    y_train_val, test_size = 0.2, random_state = 123)

In [192]:
combined.head()

,label,tweet,adjective_freq,adverb_freq,noun_freq,verb_freq
id,,,,,,
1,0.0,@user when a father is dysfunctional and is s...,0.066667,0.066667,0.400000,0.066667
2,0.0,@user @user thanks for #lyft credit i can't us...,0.000000,0.000000,0.666667,0.000000
3,0.0,bihday your majesty,0.000000,0.000000,0.600000,0.200000
4,0.0,#model i love u take with u all the time in ...,0.000000,0.250000,0.500000,0.000000
5,0.0,factsguide: society now #motivation,0.250000,0.000000,0.500000,0.000000
